In [16]:
import pandas as pd
import numpy as np
import model
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import classification_report
import numpy as np
from collections import defaultdict


In [3]:
columns = ["AccelX(g)_leg","AccelY(g)_leg","AccelZ(g)_leg","GyroX(¬∞/s)_leg","GyroY(¬∞/s)_leg","GyroZ(¬∞/s)_leg","AccelX(g)_hand","AccelY(g)_hand","AccelZ(g)_hand","GyroX(¬∞/s)_hand","GyroY(¬∞/s)_hand","GyroZ(¬∞/s)_hand","AccelX(g)_chest","AccelY(g)_chest","AccelZ(g)_chest","Sample(V)","Activity_Label"]
df = pd.read_csv("/home/jupyter-saswar/HealthySubs_Modeling/healthy_smote.csv", header=0, names=columns)


In [4]:
labels = pd.factorize(df['Activity_Label'])[0]

feature_columns = df.columns[:-1]  
data = df[feature_columns].values

In [5]:
# define input shape and activity count as per your dataset
input_shape = (128, 16)
activityCount = 33  

HART = model.HART(input_shape, activityCount)
MobileHART = model.mobileHART_XS(input_shape, activityCount)

2024-02-29 01:50:43.563458: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-29 01:50:43.563652: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-29 01:50:43.582994: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [6]:
def create_overlapping_windows(data, window_size, step_size, features):
    segments = []
    for start in range(0, len(data) - window_size + 1, step_size):
        segment = data[start:start + window_size, :features]
        segments.append(segment)
    return np.array(segments)

window_size = 128  
step_size = 64    
num_features = 16 

reshaped_data = create_overlapping_windows(data, window_size, step_size, num_features)


def assign_labels_to_windows(labels, window_size, step_size):
    window_labels = []
    for start in range(0, len(labels) - window_size + 1, step_size):
        window_label = np.bincount(labels[start:start + window_size]).argmax()
        window_labels.append(window_label)
    return np.array(window_labels)

window_labels = assign_labels_to_windows(labels, window_size, step_size)


In [7]:
=X_train, X_test, y_train, y_test = train_test_split(reshaped_data, window_labels, test_size=0.2, random_state=21)

In [8]:
HART.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [9]:
early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss',  
    patience=5,         
    restore_best_weights=True  
)

In [10]:
history = HART.fit(X_train, y_train, epochs=200, batch_size=64, validation_data=(X_test, y_test),callbacks=[early_stopping])

Epoch 1/200


2024-02-29 01:51:14.628366: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8902
2024-02-29 01:51:15.433556: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-02-29 01:51:15.574508: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-02-29 01:51:16.396121: I external/local_xla/xla/service/service.cc:168] XLA service 0x55e498ca8580 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-02-29 01:51:16.396152: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2024-02-29 01:51:16.396162: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (1): Quadro P6000, Compute Capability 6.1
2024-02-29 01:51:16.403249: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] d

533/533 [==============================] - 37s 34ms/step - loss: 0.7538 - accuracy: 0.7517 - val_loss: 0.4397 - val_accuracy: 0.8554
Epoch 2/200
533/533 [==============================] - 17s 32ms/step - loss: 0.3637 - accuracy: 0.8805 - val_loss: 0.3114 - val_accuracy: 0.8952
Epoch 3/200
533/533 [==============================] - 17s 31ms/step - loss: 0.2601 - accuracy: 0.9157 - val_loss: 0.2267 - val_accuracy: 0.9257
Epoch 4/200
533/533 [==============================] - 17s 32ms/step - loss: 0.2181 - accuracy: 0.9276 - val_loss: 0.1829 - val_accuracy: 0.9379
Epoch 5/200
533/533 [==============================] - 17s 31ms/step - loss: 0.1818 - accuracy: 0.9392 - val_loss: 0.2074 - val_accuracy: 0.9308
Epoch 6/200
533/533 [==============================] - 17s 31ms/step - loss: 0.1621 - accuracy: 0.9463 - val_loss: 0.1588 - val_accuracy: 0.9451
Epoch 7/200
533/533 [==============================] - 17s 31ms/step - loss: 0.1376 - accuracy: 0.9547 - val_loss: 0.1608 - val_accuracy: 0.94

In [11]:
performance = HART.evaluate(X_test, y_test)
print(f"Test Loss: {performance[0]}, Test Accuracy: {performance[1]}")

267/267 [==============================] - 2s 8ms/step - loss: 0.1022 - accuracy: 0.9705
Test Loss: 0.1021907776594162, Test Accuracy: 0.9705191254615784


In [12]:
y_pred_prob = HART.predict(X_test)  
y_pred = np.argmax(y_pred_prob, axis=1)  

report = classification_report(y_test, y_pred, target_names=[f"Class {i}" for i in range(activityCount)])

print(report)


267/267 [==============================] - 3s 8ms/step
              precision    recall  f1-score   support

     Class 0       0.98      0.96      0.97       242
     Class 1       0.96      0.98      0.97       277
     Class 2       1.00      1.00      1.00       263
     Class 3       1.00      0.99      0.99       241
     Class 4       0.97      0.92      0.94       241
     Class 5       0.99      0.95      0.97       235
     Class 6       0.96      1.00      0.98       263
     Class 7       0.98      0.99      0.98       262
     Class 8       0.99      0.99      0.99       257
     Class 9       0.95      0.95      0.95       262
    Class 10       0.94      0.93      0.93       261
    Class 11       1.00      1.00      1.00       259
    Class 12       1.00      0.99      0.99       278
    Class 13       0.99      1.00      0.99       282
    Class 14       0.98      0.98      0.98       245
    Class 15       0.97      0.96      0.97       243
    Class 16       0.96   

## Testing the healthy trained model on PD data

In [13]:
#read and preprocess the test data
test_columns = ["AccelX(g)_leg", "AccelY(g)_leg", "AccelZ(g)_leg", "GyroX(¬∞/s)_leg", "GyroY(¬∞/s)_leg", "GyroZ(¬∞/s)_leg", "AccelX(g)_hand", "AccelY(g)_hand", "AccelZ(g)_hand", "GyroX(¬∞/s)_hand", "GyroY(¬∞/s)_hand", "GyroZ(¬∞/s)_hand", "AccelX(g)_chest", "AccelY(g)_chest", "AccelZ(g)_chest", "Sample(V)", "Activity_Label"]
test_df = pd.read_csv("/home/jupyter-saswar/PD_smote.csv", header=0, names=test_columns)

#factorize test labels
test_labels = pd.factorize(test_df['Activity_Label'])[0]

#select relevant columns for test features
test_feature_columns = test_df.columns[:-1]  
test_data = test_df[test_feature_columns].values

test_reshaped_data = create_overlapping_windows(test_data, window_size, step_size, num_features)

test_window_labels = assign_labels_to_windows(test_labels, window_size, step_size)

test_performance = HART.evaluate(test_reshaped_data, test_window_labels)
print(f"Test Loss: {test_performance[0]}, Test Accuracy: {test_performance[1]}")


1346/1346 [==============================] - 11s 8ms/step - loss: 21.8884 - accuracy: 0.0119
Test Loss: 21.888362884521484, Test Accuracy: 0.01193766389042139


In [14]:

#evaluate the model on the test data
test_predictions = HART.predict(test_reshaped_data)
test_predictions = np.argmax(test_predictions, axis=1)

#print classification report
print(classification_report(test_window_labels, test_predictions))



1346/1346 [==============================] - 10s 8ms/step
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1388
           1       0.00      0.00      0.00      1389
           2       0.00      0.00      0.00      1389
           3       0.00      0.00      0.00      1389
           4       0.02      0.09      0.03      1387
           5       0.00      0.00      0.00      1390
           6       0.48      0.24      0.32      1389
           7       0.00      0.00      0.00      1389
           8       0.00      0.00      0.00      1389
           9       0.00      0.00      0.00      1389
          10       0.03      0.00      0.01      1387
          11       0.13      0.00      0.01      1390
          12       0.00      0.00      0.00      1390
          13       0.00      0.00      0.00      1390
          14       0.00      0.00      0.00      1389
          15       0.00      0.00      0.00      1389
          16       0.00

/home/jupyter-saswar/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/jupyter-saswar/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/jupyter-saswar/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [17]:
#calculate ratios and generate a DataFrame for detailed analysis

label_counts = defaultdict(int)
total_counts = defaultdict(int)

for i in range(len(test_window_labels)):
    actual_label = test_window_labels[i]
    predicted_label = test_predictions[i]
    label_counts[f"actual_{actual_label}_predicted_{predicted_label}"] += 1
    total_counts[f"actual{actual_label}"] += 1

data = []
for label, count in label_counts.items():
    actual_label = label.split("_")[1]
    total = total_counts[f"actual{actual_label}"]
    ratio = count / total if total > 0 else 0
    data.append([label, count, total, ratio])

results_df = pd.DataFrame(data, columns=["Label", "Count", "Total", "Ratio"])
results_df.sort_values(by="Ratio", ascending=False, inplace=True)
print(results_df)


                      Label  Count  Total     Ratio
267   actual_29_predicted_2   1389   1389  1.000000
173  actual_19_predicted_32   1388   1388  1.000000
249   actual_25_predicted_2   1389   1389  1.000000
160  actual_17_predicted_23   1387   1389  0.998560
161  actual_18_predicted_22   1382   1389  0.994960
..                      ...    ...    ...       ...
309   actual_13_predicted_8      1   1390  0.000719
229   actual_5_predicted_10      1   1390  0.000719
307  actual_12_predicted_32      1   1390  0.000719
288  actual_20_predicted_25      1   1390  0.000719
308   actual_13_predicted_2      1   1390  0.000719

[321 rows x 4 columns]


In [18]:
print('''Tweaked Classification Report for HART
      trained on Non PD - Tested on PD''')
results_df.head(20)

Tweaked Classification Report for HART
      trained on Non PD - Tested on PD


,Label,Count,Total,Ratio
267,actual_29_predicted_2,1389,1389,1.000000
173,actual_19_predicted_32,1388,1388,1.000000
249,actual_25_predicted_2,1389,1389,1.000000
160,actual_17_predicted_23,1387,1389,0.998560
161,actual_18_predicted_22,1382,1389,0.994960
246,actual_24_predicted_2,1382,1389,0.994960
123,actual_12_predicted_22,1382,1390,0.994245
75,actual_8_predicted_22,1365,1389,0.982721
218,actual_23_predicted_21,1350,1389,0.971922
136,actual_14_predicted_4,1301,1389,0.936645


##  HART Trained on non PD and finetuned on 30% PD Data 

In [20]:
columns = ["AccelX(g)_leg","AccelY(g)_leg","AccelZ(g)_leg","GyroX(¬∞/s)_leg","GyroY(¬∞/s)_leg","GyroZ(¬∞/s)_leg","AccelX(g)_hand","AccelY(g)_hand","AccelZ(g)_hand","GyroX(¬∞/s)_hand","GyroY(¬∞/s)_hand","GyroZ(¬∞/s)_hand","AccelX(g)_chest","AccelY(g)_chest","AccelZ(g)_chest","Sample(V)","Activity_Label"]
df = pd.read_csv("/home/jupyter-saswar/PD_smote.csv", header=0, names = columns)

from sklearn.preprocessing import LabelEncoder

LE = LabelEncoder()
df['Activity_Label'] = LE.fit_transform(df['Activity_Label'])

In [22]:

features = df.columns[:-1]  
labels_encoded = df['Activity_Label'].values

data_filtered = df[features].values

window_size = 128
step_size = 64
num_features = 16

reshaped_data_filtered = create_overlapping_windows(data_filtered, window_size, step_size, num_features)

window_labels_filtered = assign_labels_to_windows(labels_encoded, window_size, step_size)

X_fine_tune, X_test_new, y_fine_tune, y_test_new = train_test_split(reshaped_data_filtered, window_labels_filtered, test_size=0.7, random_state=21)

# Fine-tune the model on the 30% subset
HART.fit(X_fine_tune, y_fine_tune, epochs=200, batch_size=64, validation_split=0.2, callbacks=[early_stopping])


Epoch 1/200
162/162 [==============================] - 6s 34ms/step - loss: 0.7272 - accuracy: 0.8566 - val_loss: 0.1420 - val_accuracy: 0.9605
Epoch 2/200
162/162 [==============================] - 5s 32ms/step - loss: 0.1204 - accuracy: 0.9670 - val_loss: 0.0834 - val_accuracy: 0.9787
Epoch 3/200
162/162 [==============================] - 5s 32ms/step - loss: 0.0812 - accuracy: 0.9759 - val_loss: 0.0626 - val_accuracy: 0.9814
Epoch 4/200
162/162 [==============================] - 5s 32ms/step - loss: 0.0611 - accuracy: 0.9822 - val_loss: 0.0568 - val_accuracy: 0.9826
Epoch 5/200
162/162 [==============================] - 5s 32ms/step - loss: 0.0482 - accuracy: 0.9871 - val_loss: 0.0625 - val_accuracy: 0.9814
Epoch 6/200
162/162 [==============================] - 5s 32ms/step - loss: 0.0389 - accuracy: 0.9877 - val_loss: 0.0530 - val_accuracy: 0.9834
Epoch 7/200
162/162 [==============================] - 5s 32ms/step - loss: 0.0305 - accuracy: 0.9907 - val_loss: 0.0518 - val_accuracy:

In [23]:

# Evaluate the model on the remaining 70%
performance_new = HART.evaluate(X_test_new, y_test_new)
print(f"Test Loss: {performance_new[0]}, Test Accuracy: {performance_new[1]}")


942/942 [==============================] - 8s 8ms/step - loss: 0.0599 - accuracy: 0.9858
Test Loss: 0.05994854494929314, Test Accuracy: 0.9858327507972717


##  HART Trained on non PD and finetuned on 80% PD Data 

In [24]:
columns = ["AccelX(g)_leg","AccelY(g)_leg","AccelZ(g)_leg","GyroX(¬∞/s)_leg","GyroY(¬∞/s)_leg","GyroZ(¬∞/s)_leg","AccelX(g)_hand","AccelY(g)_hand","AccelZ(g)_hand","GyroX(¬∞/s)_hand","GyroY(¬∞/s)_hand","GyroZ(¬∞/s)_hand","AccelX(g)_chest","AccelY(g)_chest","AccelZ(g)_chest","Sample(V)","Activity_Label"]
df = pd.read_csv("/home/jupyter-saswar/HealthySubs_Modeling/healthy_smote.csv", header=0, names=columns)
labels = pd.factorize(df['Activity_Label'])[0]

feature_columns = df.columns[:-1]  
data = df[feature_columns].values

def create_overlapping_windows(data, window_size, step_size, features):
    segments = []
    for start in range(0, len(data) - window_size + 1, step_size):
        segment = data[start:start + window_size, :features]
        segments.append(segment)
    return np.array(segments)

# Define parameters
window_size = 128  
step_size = 64    
num_features = 16 

reshaped_data = create_overlapping_windows(data, window_size, step_size, num_features)


def assign_labels_to_windows(labels, window_size, step_size):
    window_labels = []
    for start in range(0, len(labels) - window_size + 1, step_size):
        window_label = np.bincount(labels[start:start + window_size]).argmax()
        window_labels.append(window_label)
    return np.array(window_labels)

window_labels = assign_labels_to_windows(labels, window_size, step_size)


X_train, X_test, y_train, y_test = train_test_split(reshaped_data, window_labels, test_size=0.2, random_state=42)

HART.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

history = HART.fit(X_train, y_train, epochs=200, batch_size=64, validation_data=(X_test, y_test), callbacks=[early_stopping])


Epoch 1/200
533/533 [==============================] - 31s 33ms/step - loss: 0.2236 - accuracy: 0.9421 - val_loss: 0.0823 - val_accuracy: 0.9760
Epoch 2/200
533/533 [==============================] - 17s 31ms/step - loss: 0.0670 - accuracy: 0.9791 - val_loss: 0.0474 - val_accuracy: 0.9854
Epoch 3/200
533/533 [==============================] - 17s 31ms/step - loss: 0.0525 - accuracy: 0.9833 - val_loss: 0.0591 - val_accuracy: 0.9823
Epoch 4/200
533/533 [==============================] - 17s 31ms/step - loss: 0.0434 - accuracy: 0.9855 - val_loss: 0.0585 - val_accuracy: 0.9837
Epoch 5/200
533/533 [==============================] - 17s 31ms/step - loss: 0.0449 - accuracy: 0.9847 - val_loss: 0.0542 - val_accuracy: 0.9821
Epoch 6/200
533/533 [==============================] - 17s 31ms/step - loss: 0.0540 - accuracy: 0.9830 - val_loss: 0.0576 - val_accuracy: 0.9826
Epoch 7/200
533/533 [==============================] - 17s 32ms/step - loss: 0.0356 - accuracy: 0.9876 - val_loss: 0.0671 - val_ac

In [25]:
columns = ["AccelX(g)_leg","AccelY(g)_leg","AccelZ(g)_leg","GyroX(¬∞/s)_leg","GyroY(¬∞/s)_leg","GyroZ(¬∞/s)_leg","AccelX(g)_hand","AccelY(g)_hand","AccelZ(g)_hand","GyroX(¬∞/s)_hand","GyroY(¬∞/s)_hand","GyroZ(¬∞/s)_hand","AccelX(g)_chest","AccelY(g)_chest","AccelZ(g)_chest","Sample(V)","Activity_Label"]
df = pd.read_csv("/home/jupyter-saswar/PD_smote.csv", header=0, names = columns)

from sklearn.preprocessing import LabelEncoder

LE = LabelEncoder()
df['Activity_Label'] = LE.fit_transform(df['Activity_Label'])

In [26]:
features = df.columns[:-1]  
labels_encoded = df['Activity_Label'].values

data_filtered = df[features].values

window_size = 128
step_size = 64
num_features = 16

reshaped_data_filtered = create_overlapping_windows(data_filtered, window_size, step_size, num_features)

window_labels_filtered = assign_labels_to_windows(labels_encoded, window_size, step_size)

X_fine_tune, X_test_new, y_fine_tune, y_test_new = train_test_split(reshaped_data_filtered, window_labels_filtered, test_size=0.2, random_state=21)

# Fine-tune the model on the 80% subset
HART.fit(X_fine_tune, y_fine_tune, epochs=200, batch_size=64, validation_split=0.2, callbacks=[early_stopping])


Epoch 1/200
431/431 [==============================] - 14s 33ms/step - loss: 0.0816 - accuracy: 0.9769 - val_loss: 0.0236 - val_accuracy: 0.9930
Epoch 2/200
431/431 [==============================] - 14s 31ms/step - loss: 0.0403 - accuracy: 0.9881 - val_loss: 0.0233 - val_accuracy: 0.9929
Epoch 3/200
431/431 [==============================] - 13s 31ms/step - loss: 0.0298 - accuracy: 0.9905 - val_loss: 0.0261 - val_accuracy: 0.9920
Epoch 4/200
431/431 [==============================] - 13s 31ms/step - loss: 0.0273 - accuracy: 0.9916 - val_loss: 0.0200 - val_accuracy: 0.9936
Epoch 5/200
431/431 [==============================] - 13s 31ms/step - loss: 0.0204 - accuracy: 0.9933 - val_loss: 0.0274 - val_accuracy: 0.9900
Epoch 6/200
431/431 [==============================] - 14s 31ms/step - loss: 0.0180 - accuracy: 0.9939 - val_loss: 0.0291 - val_accuracy: 0.9914
Epoch 7/200
431/431 [==============================] - 13s 31ms/step - loss: 0.0182 - accuracy: 0.9938 - val_loss: 0.0251 - val_ac

In [27]:

# Evaluate the model on the remaining 20%
performance_new = HART.evaluate(X_test_new, y_test_new)
print(f"Test Loss: {performance_new[0]}, Test Accuracy: {performance_new[1]}")

270/270 [==============================] - 2s 8ms/step - loss: 0.0320 - accuracy: 0.9895
Test Loss: 0.032017454504966736, Test Accuracy: 0.989549458026886


##  HART on PD Smote Data


In [28]:
columns = ["AccelX(g)_leg", "AccelY(g)_leg", "AccelZ(g)_leg", "GyroX(¬∞/s)_leg", "GyroY(¬∞/s)_leg", "GyroZ(¬∞/s)_leg", "AccelX(g)_hand", "AccelY(g)_hand", "AccelZ(g)_hand", "GyroX(¬∞/s)_hand", "GyroY(¬∞/s)_hand", "GyroZ(¬∞/s)_hand", "AccelX(g)_chest", "AccelY(g)_chest", "AccelZ(g)_chest", "Sample(V)", "Activity_Label"]
df1 = pd.read_csv("/home/jupyter-saswar/PD_smote.csv", header=0, names=columns)

labels = pd.factorize(df1['Activity_Label'])[0]

feature_columns = df1.columns[:-1] 
data = df1[feature_columns].values

In [29]:
#define parameters for overlapping windows
window_size = 128 
step_size = 64    
num_features = 16 

reshaped_data = create_overlapping_windows(data, window_size, step_size, num_features)

window_labels = assign_labels_to_windows(labels, window_size, step_size)

input_shape = (128, 16)
activityCount = len(np.unique(labels)) 


In [30]:
HART = model.HART(input_shape, activityCount)

X_train, X_test, y_train, y_test = train_test_split(reshaped_data, window_labels, test_size=0.2, random_state=42)

HART.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)


In [31]:
history = HART.fit(X_train, y_train, epochs=200, batch_size=64, validation_data=(X_test, y_test), callbacks=[early_stopping])


Epoch 1/200
539/539 [==============================] - 32s 34ms/step - loss: 0.2984 - accuracy: 0.9186 - val_loss: 0.1659 - val_accuracy: 0.9475
Epoch 2/200
539/539 [==============================] - 17s 31ms/step - loss: 0.1022 - accuracy: 0.9704 - val_loss: 0.1115 - val_accuracy: 0.9652
Epoch 3/200
539/539 [==============================] - 17s 32ms/step - loss: 0.0828 - accuracy: 0.9751 - val_loss: 0.0847 - val_accuracy: 0.9761
Epoch 4/200
539/539 [==============================] - 17s 31ms/step - loss: 0.0647 - accuracy: 0.9800 - val_loss: 0.0680 - val_accuracy: 0.9788
Epoch 5/200
539/539 [==============================] - 17s 32ms/step - loss: 0.0577 - accuracy: 0.9818 - val_loss: 0.1554 - val_accuracy: 0.9609
Epoch 6/200
539/539 [==============================] - 17s 32ms/step - loss: 0.0545 - accuracy: 0.9833 - val_loss: 0.0766 - val_accuracy: 0.9793
Epoch 7/200
539/539 [==============================] - 17s 32ms/step - loss: 0.0490 - accuracy: 0.9846 - val_loss: 0.0629 - val_ac

In [32]:
performance = HART.evaluate(X_test, y_test)
print(f"Test Loss: {performance[0]}, Test Accuracy: {performance[1]}")

270/270 [==============================] - 2s 8ms/step - loss: 0.0540 - accuracy: 0.9841
Test Loss: 0.054036714136600494, Test Accuracy: 0.9840919375419617


In [33]:
test_predictions = HART.predict(test_reshaped_data)
test_predictions = np.argmax(test_predictions, axis=1)

print(classification_report(test_window_labels, test_predictions))


1346/1346 [==============================] - 11s 8ms/step
              precision    recall  f1-score   support

           0       0.98      0.97      0.97      1388
           1       0.97      0.98      0.97      1389
           2       0.99      0.89      0.94      1389
           3       1.00      1.00      1.00      1389
           4       0.90      0.98      0.94      1387
           5       1.00      0.98      0.99      1390
           6       0.99      0.99      0.99      1389
           7       0.99      0.98      0.99      1389
           8       0.98      1.00      0.99      1389
           9       0.97      1.00      0.99      1389
          10       0.99      0.96      0.97      1387
          11       1.00      1.00      1.00      1390
          12       1.00      1.00      1.00      1390
          13       0.99      0.99      0.99      1390
          14       1.00      1.00      1.00      1389
          15       1.00      1.00      1.00      1389
          16       1.00